In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
train = pd.read_csv('cleaned_dataset/train.csv')
test = pd.read_csv('cleaned_dataset/test.csv')

In [3]:
train['species_PIPIENS/RESTUANS']

0        1
1        1
2        1
3        1
4        1
        ..
10501    1
10502    1
10503    1
10504    1
10505    1
Name: species_PIPIENS/RESTUANS, Length: 10506, dtype: int64

In [4]:
X = train.drop(['wnvpresent', 'nummosquitos', 'species_PIPIENS/RESTUANS'], axis=1)
y = train['species_PIPIENS/RESTUANS']

In [5]:
y.value_counts()

1    10191
0      315
Name: species_PIPIENS/RESTUANS, dtype: int64

In [6]:
X

,block,latitude,longitude,addressaccuracy,tmax_stn1,tmin_stn1,tavg_stn1,depart_stn1,dewpoint_stn1,wetbulb_stn1,...,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903,street_others,trap_others
0,41,41.954690,-87.800991,9,88,60,74,10,58,65,...,0,0,0,0,0,0,0,0,0,0
1,41,41.954690,-87.800991,9,88,60,74,10,58,65,...,0,0,0,0,0,0,0,0,0,0
2,62,41.994991,-87.769279,9,88,60,74,10,58,65,...,0,0,0,0,0,0,0,0,0,0
3,79,41.974089,-87.824812,8,88,60,74,10,58,65,...,0,0,0,0,0,0,0,0,0,0
4,79,41.974089,-87.824812,8,88,60,74,10,58,65,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10501,51,41.763733,-87.742302,8,75,50,63,3,52,58,...,0,0,0,0,0,0,0,0,0,0
10502,58,41.987280,-87.666066,8,75,50,63,3,52,58,...,0,0,0,0,0,0,0,0,0,0
10503,17,41.912563,-87.668055,9,75,50,63,3,52,58,...,1,0,0,0,0,0,0,0,0,0
10504,71,42.009876,-87.807277,9,75,50,63,3,52,58,...,0,1,0,0,0,0,0,0,0,0


In [7]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='auto')

In [8]:
# Create an instance of StandardScaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Transform the data using the scaler
scaled_X = scaler.fit_transform(X)
species = test['species']
test.drop('species', axis=1, inplace =True)
test = test.reindex(columns=X.columns)
scaled_test = scaler.transform(test)

# Print the scaled data
print(scaled_X)

[[0.35227273 0.83171413 0.32553085 ... 0.         0.         0.        ]
 [0.35227273 0.83171413 0.32553085 ... 0.         0.         0.        ]
 [0.59090909 0.93981246 0.4049379  ... 0.         0.         0.        ]
 ...
 [0.07954545 0.71871798 0.65840345 ... 0.         0.         0.        ]
 [0.69318182 0.9797381  0.30979067 ... 0.         0.         0.        ]
 [0.36363636 0.35356662 0.76096504 ... 0.         0.         0.        ]]


In [9]:
X.head(1)

,block,latitude,longitude,addressaccuracy,tmax_stn1,tmin_stn1,tavg_stn1,depart_stn1,dewpoint_stn1,wetbulb_stn1,...,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903,street_others,trap_others
0,41,41.95469,-87.800991,9,88,60,74,10,58,65,...,0,0,0,0,0,0,0,0,0,0


In [10]:
test.head(1)

,block,latitude,longitude,addressaccuracy,tmax_stn1,tmin_stn1,tavg_stn1,depart_stn1,dewpoint_stn1,wetbulb_stn1,...,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903,street_others,trap_others
0,41,41.95469,-87.800991,9,86,61,74,7,56,64,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#X = train.drop(['nummosquitos', 'wnvpresent'], axis=1)
#y = train['nummosquitos']

In [12]:
X_resampled, y_resampled = smote.fit_resample(scaled_X, y)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.30)

In [16]:
# Define the models
#model_lr = LogisticRegression(C=10)
#model_nb = MultinomialNB()
#model_svc = SVC()
#model_rf = RandomForestClassifier()
#model_dt = DecisionTreeClassifier()
#model_knn = KNeighborsClassifier()
#model_gb = GradientBoostingClassifier()
#model_ab = AdaBoostClassifier()
model = XGBClassifier()


In [17]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Train score is {model.score(X_train, y_train)}, Test score is {model.score(X_test, y_test)}')
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, columns = ['Pred 0', 'Pred 1'], index = ['Actual 0','Actual 1'])
display(cm_df)
print(classification_report(y_test, y_pred))

: 

: 

In [ ]:
model.predict(scaled_test)

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
unique_values, value_counts = np.unique(model_xgb.predict(scaled_test), return_counts=True)

for value, count in zip(unique_values, value_counts):
    print(f"{value}: {count}")

0: 50811
1: 65482


In [ ]:
species.value_counts()

NOT PIPIENS/RESTUANS    57398
PIPIENS/RESTUANS        44550
others                  14345
Name: species, dtype: int64

In [ ]:
species.to_frame()

,species
0,PIPIENS/RESTUANS
1,PIPIENS/RESTUANS
2,PIPIENS/RESTUANS
3,NOT PIPIENS/RESTUANS
4,NOT PIPIENS/RESTUANS
...,...
116289,NOT PIPIENS/RESTUANS
116290,NOT PIPIENS/RESTUANS
116291,others
116292,NOT PIPIENS/RESTUANS


In [ ]:
species

0                                           PIPIENS/RESTUANS
1                                           PIPIENS/RESTUANS
2                                           PIPIENS/RESTUANS
3                                       NOT PIPIENS/RESTUANS
4                                       NOT PIPIENS/RESTUANS
                                 ...                        
116289                                  NOT PIPIENS/RESTUANS
116290                                  NOT PIPIENS/RESTUANS
116291                                                others
116292                                  NOT PIPIENS/RESTUANS
predict    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
Name: species, Length: 116294, dtype: object

In [ ]:
test_export = pd.read_csv('Kaggle Datasets/test.csv/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Kaggle Datasets/test.csv/test.csv'

In [ ]:
test_export = test_export['Id'].to_frame()

In [ ]:
test = test.reindex(columns=X.columns)

In [ ]:
test = scaler.transform(test)

In [ ]:
test_export['WnvPresent'] = model_xgb.predict_proba(test)[:, 1]

In [ ]:
test_export

In [ ]:
test_export.to_csv('test_export.csv', index=False)